In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
plt.style.use('seaborn-whitegrid')

In [ ]:
df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', len(df.columns))

In [ ]:
df.shape

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
for i in df.columns:
    print('{}:- {}\n{}\n\n'.format(i, df[i].nunique(), df[i].unique()))

In [ ]:
# Distributions of columns 
for i in df.columns:
    plt.hist(df[i], bins = 30, color = 'steelblue', edgecolor = 'black', rwidth = 0.8, alpha = 0.7)
    plt.title('{} Distribution\n'.format(i.title()), fontsize = 25)
    plt.xlabel('\n{}'.format(i.title()), fontsize = 20)
    plt.ylabel('Frequency\n', fontsize = 20)
    plt.show()

## Working on outliers

In [ ]:
class Outliers(object):
    def __init__(self, df, col):
        self.df = df
        self.col = col
        
        self.mean = df[col].mean()
        self.median = df[col].median()
        self.min = df[col].min()
        self.max = df[col].max()
        self.std = df[col].std()
        self.quantile_25 = self.df[self.col].quantile(0.25)
        self.quantile_75 = self.df[self.col].quantile(0.75)
        
    def info(self):
        print('Mean:- {}'.format(self.mean))
        print('Median:- {}'.format(self.median))
        print('Standard Deviation:- {}'.format(self.std))
        print('Minimum value:- {}'.format(self.min))
        print('Maximum value:- {}'.format(self.max))
        print('25th quantile:- {}'.format(self.quantile_25))
        print('75th quantile:- {}'.format(self.quantile_75))
        
        des = '*' * 20
        return des
        
        
class IQR(Outliers):
    def __init__(self, df, col):
        super().__init__(df, col)
        
    def iqr_calc(self):
        
        IQR = self.quantile_75 - self.quantile_25
        
        lower_bound = self.quantile_25 - (1.5 * IQR)
        upper_bound = self.quantile_75 + (1.5 * IQR)
        
        return lower_bound, upper_bound
        
    def iqr_outliers(self):
        
        lower_bound, upper_bound = self.iqr_calc()
        return self.df.loc[(self.df[self.col] < lower_bound) | (self.df[self.col] > upper_bound), self.col]
        
    def iqr_remove(self):
        
        lower_bound, upper_bound = self.iqr_calc()
        return self.df.loc[(self.df[self.col] > lower_bound) & (self.df[self.col] < upper_bound)]
        
    

class Z_score(Outliers):
    def __init__(self, df, col):
        super().__init__(df, col)
        
    def z_score_outliers(self):
        outlier = []
        for i in self.df[self.col]:
            z = (i - self.mean) / self.std
            if abs(z) > 3:
                outlier.append(i)
        
        return outlier
    
    def z_score_remove(self):
        
        df_copy = self.df.copy()
        for i in self.z_score_outliers():
            df_copy = df_copy.loc[df_copy[self.col] != i]
            
        return df_copy
    

class StandardDeviation(Outliers):
    def __init__(self, df, col):
        super().__init__(df, col)
        
    def std_calc(self):
        
        lower_std = self.mean - (3 * self.std)
        upper_std = self.mean + (3 * self.std)
        
        return lower_std, upper_std
        
    def std_outliers(self):
        
        lower_std, upper_std = self.std_calc()
        return self.df.loc[(self.df[self.col] < lower_std) | (self.df[self.col] > upper_std), self.col]
    
    def std_remove(self):
        
        lower_std, upper_std = self.std_calc()
        return self.df.loc[(self.df[self.col] > lower_std) & (self.df[self.col] < upper_std)]
        
        
        

In [ ]:
columns = ['chol', 'trtbps', 'thalachh', 'oldpeak']


for i in columns:
    out = Outliers(df, i)
    iqr = IQR(df, i)
    z_score = Z_score(df, i)
    std = StandardDeviation(df, i)    

    print('Outliers:- {}\n'.format(i))

    print('Info:- \n')
    out.info()
    print('*'*40)
    print('\n')
    
    
    print('IQR Outliers:- \n{}\n'.format(iqr.iqr_outliers())) 
    print('Shape of df if removed outliers with IQR:- {}'.format(iqr.iqr_remove().shape))
    print('*'*20)
    
    print('Z-score Outliers:- \n{}\n'.format(z_score.z_score_outliers())) 
    print('Shape of df if removed outliers with Z-score:- {}'.format(z_score.z_score_remove().shape))
    print('*'*20)
    
    print('StandardDeviation Outliers:- \n{}\n'.format(std.std_outliers())) 
    print('Shape of df if removed outliers with StandardDeviation:- {}'.format(std.std_remove().shape))
    
    print('\n', '*'*100, '\n', '*'*100, '\n', '*'*100, '\n')
    

In [ ]:
##### Treating Outliers of chol
z_score_chol = Z_score(df, 'chol')
df = z_score_chol.z_score_remove()

In [ ]:
##### Treating Outliers of trtbps
std_trtbps = StandardDeviation(df, 'trtbps')
df = std_trtbps.std_remove()

In [ ]:
##### Treating Outliers of thalachh
iqr_thalachh = IQR(df, 'thalachh')
df = iqr_thalachh.iqr_remove()

In [ ]:
##### Treating Outliers of oldpeak
z_score_oldpeak = Z_score(df, 'oldpeak')
df = z_score_oldpeak.z_score_remove()

In [ ]:
df.shape

In [ ]:
# Distributions of columns aftet treating outliers
for i in df.columns:
    plt.hist(df[i], bins = 30, color = 'steelblue', edgecolor = 'black', rwidth = 0.8, alpha = 0.7)
    plt.title('{} Distribution\n'.format(i.title()), fontsize = 25)
    plt.xlabel('\n{}'.format(i.title()), fontsize = 20)
    plt.ylabel('Frequency\n', fontsize = 20)
    plt.show()

## EDA

In [ ]:
sex_out = df.groupby(['sex', 'output'])[['output']].count()

#idx = pd.IndexSlice
#sex_out.loc[idx[0, :], :].output

indx = np.arange(len(sex_out.unstack().output.index))
width = 0.25


plt.bar(indx - width/2, sex_out.unstack().output[0].values,
        width = width, alpha = 0.7, edgecolor = 'black', label = 'less chance of heart attack')
plt.bar(indx + width/2, sex_out.unstack().output[1].values,
        width = width, alpha = 0.7, edgecolor = 'black', label = 'more chance of heart attack')

plt.title('Frequency of people with chance of heart attack \n(gender wise)\n', fontsize = 25)
plt.xlabel('\nSex', fontsize = 20)
plt.ylabel('Frequency\n', fontsize = 20)


plt.xticks([0,1], ['Female', 'Male'])
plt.legend(frameon = True, facecolor = 'gray', framealpha = 0.2, shadow = False, fontsize = 12.5)
plt.show()

In [ ]:
sex_atk = df.groupby('sex')[['output']].mean()

plt.bar(sex_atk.index.map({0: 'Female', 1: 'Male'}), sex_atk.output, color = 'red', 
        alpha = 0.5, edgecolor = 'black', width = 0.5)

plt.title('Chance of getting heart attack of each gender type\n', fontsize = 25)
plt.xlabel('\nChance of getting heart attack (in %)', fontsize = 20)
plt.ylabel('Genders\n', fontsize = 20)
plt.show()

In [ ]:
sex_hrt = df.groupby('sex')[['output']].mean()

plt.pie(sex_hrt.output, labels = sex_hrt.index.map({0: 'Female', 1: 'Male'}), wedgeprops = {'edgecolor': 'black'}, 
        shadow = True, textprops = {'fontsize': 15}, autopct = '%1.2f%%')
plt.title('Chance of getting (if compared according to genders)', fontsize = 20)
plt.show()

In [ ]:
sex_info = df.groupby('sex')[['chol', 'trtbps', 'thalachh', 'oldpeak']].aggregate(np.mean)

plt.figure(figsize = (9, 5))

indx = np.arange(len(sex_info.columns))
width = 0.2

plt.bar(indx - width/2, sex_info.loc[0, :], width = width, alpha = 0.5, color = 'pink', label = 'Female', edgecolor = 'black')
plt.bar(indx + width/2, sex_info.loc[1, :], width = width, alpha = 0.5, color = 'blue', label = 'Male', edgecolor = 'black')

plt.title('Comparing genders on avg. values of different categories\n', fontsize = 25)
plt.xlabel('\nCategories', fontsize = 20)
plt.ylabel('Average values\n', fontsize = 20)

plt.yscale('log')
plt.xticks([0, 1, 2, 3], ['chol', 'trtbps', 'thalachh', 'oldpeak'])
plt.legend(loc = 'best', frameon = True, fontsize = 15)
plt.show()

In [ ]:
age_dis = pd.qcut(df.age, 5)
age_hrt = df.pivot_table('output', index = age_dis, columns = 'sex')

plt.figure(figsize = (12,9))

indx = np.arange(len(age_hrt.index))
width = 0.4

plt.barh(indx - width/2, age_hrt[0], height = width, alpha = 0.7, edgecolor = 'black', label = 'Female')
plt.barh(indx + width/2, age_hrt[1], height = width, alpha = 0.7, edgecolor = 'black', label = 'Male')

plt.title('Chances of having heart attack as pereach age group\n', fontsize = 25)
plt.xlabel('\nChances of heart attack (in %)', fontsize = 20)
plt.ylabel('Age Group\n', fontsize = 20)

plt.yticks(indx, age_hrt.index)
plt.legend(fontsize = 15, frameon = True, loc = 'best')
plt.show()

## Creating Model

In [ ]:
df.head()

In [ ]:
for i in df.columns:
    print(i, ':- ', df[i].nunique())

In [ ]:
X = df.drop('output', axis = 1).copy()
y = df['output']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

In [ ]:
cat1 = ['sex', 'fbs','exng']
cat2 = ['cp', 'restecg', 'caa', 'thall']
num = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']

In [ ]:
trans_col = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore'), cat2), 
                                    (OrdinalEncoder(), cat1), 
                                    (StandardScaler(), num), 
                                    remainder = 'passthrough')

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipe_lr = make_pipeline(trans_col, LogisticRegression(solver = 'liblinear'))
pipe_svm = make_pipeline(trans_col, SVC())
pipe_rf = make_pipeline(trans_col, RandomForestClassifier())

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params_lr = {
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__tol': [0.01, 0.001, 0.0001, 0.00001]
}

grid_lr = GridSearchCV(estimator = pipe_lr, param_grid = params_lr, cv = 10, verbose = 10)

In [ ]:
params_svm = {
    'svc__kernel': ['linear', 'rbf'],
    'svc__C': [1, 10, 100, 1000, 10000]
}

grid_svm = GridSearchCV(estimator = pipe_svm, param_grid = params_svm, cv = 10, verbose = 10)

In [ ]:
params_rf = {
    'randomforestclassifier__criterion': ['entropy', 'gini'],
    'randomforestclassifier__max_depth': [2, 4, 6, 8, 10],
    'randomforestclassifier__min_samples_split': [2, 4, 6, 8, 10],
    'randomforestclassifier__min_samples_leaf': [2, 3, 4, 5]
}

grid_rf = GridSearchCV(estimator = pipe_rf, param_grid = params_rf, cv = 10, verbose = 10, n_jobs = -1)

In [ ]:
grid_lr.fit(X_train, y_train)

In [ ]:
grid_svm.fit(X_train, y_train)

In [ ]:
grid_rf.fit(X_train, y_train)

In [ ]:
print(grid_lr.best_params_)
print(grid_lr.score(X_train, y_train))
print(grid_lr.score(X_test, y_test))

In [ ]:
print(grid_svm.best_params_)
print(grid_svm.score(X_train, y_train))
print(grid_svm.score(X_test, y_test))

In [ ]:
print(grid_rf.best_params_)
print(grid_rf.score(X_train, y_train))
print(grid_rf.score(X_test, y_test))

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
print(f1_score(y_test, grid_lr.predict(X_test)))
print(f1_score(y_test, grid_svm.predict(X_test)))
print(f1_score(y_test, grid_rf.predict(X_test)))